In [1]:
## Notebook property setup.
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf, col
from pyspark import SparkContext
from pyspark.sql import SparkSession

import sys
import time
import os.path
import json
from datetime import datetime
from operator import add

from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import preprocessor as p
import string

## Enable inline graphs
%matplotlib inline

## Display precision for pandas dataframe
pd.set_option('precision',10)

## Set up language classifier, used to filtered out non-English files
import langid
langid.set_languages(['de','fr','it','en','zh','ar','ja','ko', 'es','ms','tr','hi','bn','pa'])

workdir = "parquet_out"

## Reading DATA

###### Raw data are saved as json.gz format. We need to load and parse these data into spark RDD. Note that, the sc.textFile function's input directory could be either a file or a directory. Spark context will create partitions automatically. 

In [4]:
# 2013 data
sc =SparkContext()

data_2013_raw = sc.textFile("twitter_data/2013-07-*")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-1e444d5cce5c>:2 

In [5]:
data_2013_raw.take(1)

[u'{"created_at":"Sun Jun 30 17:57:53 +0000 2013","id":351399204360425472,"id_str":"351399204360425472","text":"I thought we had something but I guess not","source":"\\u003ca href=\\"http:\\/\\/twitter.com\\/download\\/iphone\\" rel=\\"nofollow\\"\\u003eTwitter for iPhone\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":950790356,"id_str":"950790356","name":"Coby Pritchett","screen_name":"CobyPritchett","location":"","url":null,"description":null,"protected":false,"followers_count":54,"friends_count":364,"listed_count":0,"created_at":"Fri Nov 16 01:45:21 +0000 2012","favourites_count":49,"utc_offset":null,"time_zone":null,"geo_enabled":false,"verified":false,"statuses_count":11,"lang":"en","contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\\/\\/a0.twimg.com\\/imag

In [6]:
# Helper function to keep track the run time of a spark ops.
def getTime(start):
    sec = time.time() - start
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    print('Spark operation takes - %d:%02d:%02d which is %d seconds in total' % (h,m,s,sec))
    
    
# Remove invalid tweet which has length less than 1000.
def ValidJson(d):
    return len(d) > 1000

# load json object, if a line is invalid, substitute as an empty dict (which has len() == 0 )
def loadJson(d):
    try:
        js = json.loads(d)
        
    except ValueError as e:
        js = {}
        
    except Exception:
        js = {}
        
    return js

# Some tweet does not contain the 'lang' key, removing as invalid.
def containsLang(d):
    return 'lang' in d

# Raw filter using twitter's default language detection. Note that the accuracy is very low, therefore we need to apply a 
# second level language detection to further remove non-Eng tweets. 
def Eng_Label(d):
    return d['lang'] == 'en'

# Convert timestamp to unix time string, usful when finding hashtag bithdates later.
def getUnixTimeStamp(stamp):
    d = datetime.strptime(stamp,'%a %b %d %H:%M:%S +0000 %Y')
    unixtime = time.mktime(d.timetuple())
    return unixtime


# Parse out the releavant attributes from raw tweet to save memory,also converting hastags and mentions to space-separate lists.
def RawParser(d):
    processed = {"from_user":d['user']['screen_name'],
                 "from_id":d['user']['id'],
                 ## Split hashtag, we only want the text in hashtag, discard indices.
                 "tweet_id":d['id'],
                 "hashtag":" ".join([hash_string['text'] for hash_string in d['entities']['hashtags']]), 
                 ## Split terms in tweet text, remove \n and \r
                 "term": d['text'],
                 ## append loc_ to each word in location
                 #"location":['loc_' + s for s in d['user']['location'].split(" ")],
                 "location":d['user']['location'],
                 ## mention ids
                 "mention":" ".join([mention['screen_name'] for mention in d['entities']['user_mentions']]),
                 "create_time":getUnixTimeStamp(d['created_at'])
                }
    return processed



In [7]:
FeatureRDD_2013 = data_2013_raw.filter(ValidJson).map(loadJson).filter(lambda x: len(x) > 1).filter(containsLang).filter(Eng_Label).map(RawParser)

In [9]:
print("Year 2013 contains "+ str(FeatureRDD_2013.getNumPartitions())+" file partitions")

Year 2013 contains 57 file partitions


In [11]:
## Define Dataframe schema. Converting RDD to dataframe.
sqlContext = SQLContext(sc)

schema = StructType([StructField('create_time', DoubleType(), False),
                     StructField('from_id', StringType(), False),
                     StructField('from_user', StringType(), False),
                     StructField('hashtag', StringType(), True),
                     StructField('location', StringType(), True),
                     StructField('mention', StringType(), True),
                     StructField('term', StringType(), True),
                     StructField('tweet_id', StringType(), False)
                    ])
Feature_df_2013 = sqlContext.createDataFrame(FeatureRDD_2013, schema)

In [12]:
Feature_df_2013.show(2)

+-------------+---------+-------------+-------+--------------------+-------+--------------------+------------------+
|  create_time|  from_id|    from_user|hashtag|            location|mention|                term|          tweet_id|
+-------------+---------+-------------+-------+--------------------+-------+--------------------+------------------+
|1.372629473E9|950790356|CobyPritchett|       |                    |       |I thought we had ...|351399204360425472|
|1.372629473E9| 55845061|DeMelloGandhi|       |Catalunya , Barce...|       |how about we smok...|351399204356231169|
+-------------+---------+-------------+-------+--------------------+-------+--------------------+------------------+
only showing top 2 rows



In [13]:
#OPTIONAL: Saving parsed 2013 data to parquet, save space, better performance
Feature_df_2013.write.save(workdir+"/2013_Raw_Eng.parquet", format="parquet")

In [19]:
test = sqlContext.read.parquet(workdir+"/2013_Raw_Eng.parquet")

Py4JJavaError: An error occurred while calling o193.parquet.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.collection.mutable.HashMap.foreachEntry(HashMap.

#### Reading parquet into rdd again for non-english filter.

In [17]:
#Ref: https://www.mail-archive.com/user@spark.apache.org/msg28820.html    changing user permission.
All_RDD2013 = sqlContext.read.parquet(workdir+"/2013_Raw_parquet")

AnalysisException: u'Path does not exist: file:/home/reda/NetBeansProjects/TwitterSensor/parquet_out/2013_Raw_parquet;'

##### Saving dataframes above as an intermediate json file. Unless you need additional attributes, this should be the data the you work with for later processing steps. the raw data is not longer releavant at this time.
##### It is much more difficult to perfrom custom map reduce on dataframe; it is easier to work with RDDs. Also, It is easier to save data as json than load into rdd comparing to converting dataframe to RDD directly (will get Row type, not primative RDD).  Therefore, we save the same data as json format as well.

In [ ]:
#All_RDD2013.write.json(workdir_eng+"/2013_Raw")

In [ ]:
#All_RDD2014.write.json(workdir_eng+"/2014_Raw")

# Utilizing the langid package to filter out tweets which contains non english char in tweet terms.

##### There are various lanuage detection libraries for python, the problem comes down to speed and accuracy. Out of all packages I tested (apache Tika, langid, lang detect, guess-language, textblob), two stand out the most: langid and textblob. 

##### Langid utilize multithreading, and works great on short text (ex. tweet terms),however, the accuracy decreases when multiple languages are mixed up in the text. It takes 0.0003 seconds to check one line. It has a major drawback: the multi-threading module in this package does not seem to work well with spark. In other words, if we try to concurrently run multiple python jobs with this lib, it will create deadlocks. It will be interesting to look into the source code of this lib to understand why.

##### Textblob is based on NLTK, and it delivers the best accuracy among other packages available. When multiple lanuage appears in the text, the majority wins (bayesian). However, it takes an average of 0.1 seconds to process one line, which is way too slow for big data practice.

##### considering accuracy and efficiency, I used Langid here. Since it does not run well with spark, I create a separate bash script to trigger 2 python instance to process data for 2013 and 2014. 

#### Run this command to execute 2013_Eng_Filter.py and 2014_Eng_Filter.py. It takes 5 full days to finish the parsing. (if line magic does not work, run it in terminal instead.) Of course, you can distribute the json file to multiple disk and run multi-thread python to save time. But consider this is a one time process, it should not matter too much.

In [ ]:
## !bash Eng_Filter.sh

##### This script will filter the json data and return English tweets only. note that, the location and hashtag may still contain non-english words.

In [ ]:
# 2013 data
data_2013_Eng = sc.textFile("/mnt/1e69d2b1-91a9-473c-a164-db90daf43a3d/Eng_Json/*")

In [ ]:
# 2014 data
data_2014_Eng = sc.textFile("/mnt/2b53fde0-61da-4eeb-a038-9910540ff9ad/Eng_Json/*")

#### 2013 data contains 360048691 valid english tweets, 2014 data contains 455285530 valid engish tweets. We have a total of 815,334,221 comapring to 829, 026, 458 in the Paper.

# Now we have English data, let's go back to the ETL process to do some feature engineering.